# DTSC-580: Data Manipulation
## Assignment: Survivor

### Name:

## Overview

In this assignment, you will test all the skills that you have learned during this course to manipulate the provided data to find the answers to questions about the TV show Survivor.  If you are not familiar with this show, start by watching this short clip that briefly explains it:  [Survivor Explained](https://www.youtube.com/watch?v=l1-hTpG_krk)

Please note that your notebook should be named `survivor` when submitting to CodeGrade for the automatic grading to work properly.

## Data

The Survivor data is a R package from Daniel Oehm.  Daniel has made the data for this package available as an Excel file as explained in his article on [gradientdescending.com](http://gradientdescending.com/survivor-data-from-the-tv-series-in-r/).  Please make sure that you use the file from our Brightspace page though to make sure that your data will match what CodeGrade is expecting.  We have also updated some errors in the file, which is another reason that you must use the data given to you.

You need to first read the article on the website linked above.  This will give you additional details about the data that will be important as you answer the questions below.

Please note that there is a data dictionary in the file that explains the columns in the data.  You will also want to become familiar with the various spreadsheets and column names.  This will help you out tremendously in this assignment.

Finally, here are a couple of things to know for those of you that have not seen the show:
- Survivor is a reality TV show that first aired May 31, 2000 and is currently still on TV.
- Contestants are broken up into two teams (usually) where they live in separate camps. 
- The teams compete in various challenges for rewards (food, supplies, brief experience trips, etc) and tribal immunity.  
- The team that loses a challenge, and therefore doesn't get the tribal immunity, goes to tribal council where they have to vote one of their members out (this data is represented in the "Vote History" spreadsheet).
- After there are a small number of contestants left, the tribes are merged into one tribe where each contestant competes for individual immunity.  The winner of the individual immunity cannot get voted out and is safe at the next tribal council.
- The are also hidden immunity idols that are hidden around the campground.  If a contestant finds and plays their hidden immunity at the tribal council, then all votes against them do not count, and the player with the next highest number of votes goes home.
- When the contestants get down to 2 or 3 people, a number of the last contestants, known as the jury, come back to vote for the person who they think should win the game.  The winner is the one who gets the most jury votes (this data is represented in the "Jury Votes" spreadsheet).  This person is known as the Sole Survivor.
- Voting recap: 
    - Tribal Council votes (Vote History spreadsheet) are bad; contestants with the most votes get sent home
    - Jury Votes (Jury Votes spreadsheet) are good; contestants with the most votes win the game and is the Sole Survivor

## Note

<u>Show Work</u>

Remember that you must show your work.  Students submissions are spot checked manually to verify that they are not hard coding the answer from looking only in the file or in CodeGrade's expected output.  If this is seen, the student's answer will be manually marked wrong and their grade will be changed to reflect this.

For example, if the question is who is the contestant who has received the most tribal votes to be voted out.  Select their record from the `castaway_details` DataFrame.  

You would show your work and code similar to this:
```
### incorrect way ###
Q1 = castaway_details[castaway_details['castaway_id'] ==  333]

### correct way - showing your work ###
# get index
idx = vote_history.groupby('vote_id').size().sort_values(ascending=False).index[0]

# select row based on index 
Q1 = castaway_details[castaway_details['castaway_id'] ==  idx]
```

<u>Use Copy</u>

Don't change any of the original DataFrames unless specifically asked or CodeGrade will not work correctly for this assignment.  Make sure you use `copy()` if needed.

In [1]:
# standard imports
import pandas as pd
import numpy as np
#from openpyxl import workbook

# Do not change this option; This allows the CodeGrade auto grading to function correctly
pd.set_option('display.max_columns', None)

First, import the data from the `survivor.xlsx` file, calling the respective DataFrames the same as the sheet name but with lowercase and [snake case](https://en.wikipedia.org/wiki/Snake_case).  For example, the sheet called `Castaway Details` should be saved as a DataFrame called `castaway_details`.  Make sure that the data files are in the same folder as your notebook.

Note:  You may or may not need to install [openpyxl](https://openpyxl.readthedocs.io/en/stable/) for the code below to work.  You can use: `$ pip install openpyxl`

In [2]:
# import data from Excel

# setup Filename and Object
fileName = "survivor.xlsx"
xls = pd.ExcelFile(fileName)

# import individual sheets
castaway_details = pd.read_excel(xls, 'Castaway Details')
castaways = pd.read_excel(xls, 'Castaways')
challenge_description = pd.read_excel(xls, 'Challenge Description')
challenge_results = pd.read_excel(xls, 'Challenge Results')
confessionals = pd.read_excel(xls, 'Confessionals')
hidden_idols = pd.read_excel(xls, 'Hidden Idols')
jury_votes = pd.read_excel(xls, 'Jury Votes')
tribe_mapping = pd.read_excel(xls, 'Tribe Mapping')
viewers = pd.read_excel(xls, 'Viewers')
vote_history = pd.read_excel(xls, 'Vote History')
season_summary = pd.read_excel(xls, 'Season Summary')
season_palettes = pd.read_excel(xls, 'Season Palettes')
tribe_colours = pd.read_excel(xls, 'Tribe Colours')

**Exercise1:** Change every column name of every DataFrame to lowercase and snake case.  This is a standard first step for some programmers as lowercase makes it easier to write and snake case makes it easier to copy multiple-word column names.

For example, `Castaway Id` should end up being `castaway_id`.  You should try doing this using a `for` loop instead of manually changing the names for each column.  It should take you no more than a few lines of code.  Use stackoverflow if you need help.

In [3]:
sheets = [castaway_details, castaways, challenge_description, challenge_results, confessionals, hidden_idols, jury_votes,
         tribe_mapping, viewers, vote_history, season_summary, season_palettes, tribe_colours]
for sheet in sheets:
    #sheet.rename(columns = lambda col: col.replace(' ', '_').lower(), inplace = True)
    sheet.columns = sheet.columns.str.lower().str.replace(' ', '_')
    #sheet.columns = map(str.lower, sheet.columns)

In [4]:
castaway_details.head(2)

,castaway_id,full_name,short_name,date_of_birth,date_of_death,gender,race,ethnicity,occupation,personality_type
0,1,Sonja Christopher,Sonja,1937-01-28,NaT,Female,NaN,NaN,Musician,ENFP
1,2,B.B. Andersen,B.B.,1936-01-18,2013-10-29,Male,NaN,NaN,Real Estate Developer,ESTJ


**Q2:** What contestant was the oldest at the time of their season?  We want to look at their age at the time of the season and NOT their current age.  Select their row from the `castaway_details` DataFrame and save this as `Q2`.  This should return a DataFrame and the index and missing values should be left as is.

In [5]:
mer1 = pd.merge(castaways, castaway_details, left_on = 'castaway_id', right_on = 'castaway_id')
q2 = mer1[mer1['age'] == mer1['age'].max()]
Q2 = castaway_details[castaway_details['castaway_id'] == 14]
Q2

,castaway_id,full_name,short_name,date_of_birth,date_of_death,gender,race,ethnicity,occupation,personality_type
13,14,Rudy Boesch,Rudy,1928-01-20,2019-11-01,Male,NaN,NaN,Retired Navy SEAL,ISTJ


**Q3:** What contestant played in the most number of seasons? Select their row from the `castaway_details` DataFrame and save this as `Q3`.  This should return a DataFrame and the index and missing values should be left as is.

In [61]:
merCasts = pd.merge(castaways, castaway_details, left_on = 'castaway_id', right_on = 'castaway_id')
onePerSzn = merCasts.drop_duplicates(['season', 'castaway_id'])
#onePerSzn['full_name_x'].value_counts()
Q3 = castaway_details[castaway_details['full_name'] == 'Rob Mariano']
Q3

,castaway_id,full_name,short_name,date_of_birth,date_of_death,gender,race,ethnicity,occupation,personality_type
54,55,Rob Mariano,Boston Rob,1975-12-25,NaT,Male,NaN,NaN,Construction Worker,ESTJ


**Q4:** Create a DataFrame of all the contestants that won their season (aka their final result in the `castaways` DataFrame was the 'Sole Survivor').  Call this DataFrame `sole_survivor`.  Note that contestants may appear more than one time in this DataFrame if they won more than one season.  Make sure that the index goes from 0 to n-1 and that the DataFrame is sorted ascending by season number.

The DataFrame should have the same columns, and the columns should be in the same order, as the `castaways` DataFrame.

In [7]:
sole_survivor = castaways[castaways['result'] == 'Sole Survivor'].sort_values(by = 'season').reset_index(drop = True)
sole_survivor

,season_name,season,full_name,castaway_id,castaway,age,city,state,personality_type,episode,day,order,result,jury_status,original_tribe,swapped_tribe,swapped_tribe_2,merged_tribe,total_votes_received,immunity_idols_won
0,Survivor: Borneo,1,Richard Hatch,16,Richard,39,Newport,Rhode Island,ENTP,14,39,16,Sole Survivor,NaN,Tagi,NaN,NaN,Rattana,6,4
1,Survivor: The Australian Outback,2,Tina Wesson,32,Tina,40,Knoxville,Tennessee,ESFJ,16,42,16,Sole Survivor,NaN,Ogakor,NaN,NaN,Barramundi,0,2
2,Survivor: Africa,3,Ethan Zohn,48,Ethan,27,Lexington,Massachusetts,ISFP,15,39,16,Sole Survivor,NaN,Boran,Boran,NaN,Moto Maji,0,4
3,Survivor: Marquesas,4,Vecepia Towery,64,Vecepia,36,Hayward,California,ISTJ,15,39,16,Sole Survivor,NaN,Maraamu,Rotu,NaN,Soliantu,2,4
4,Survivor: Thailand,5,Brian Heidik,80,Brian,34,Quartz Hill,California,ISTP,15,39,16,Sole Survivor,NaN,Chuay Gahn,NaN,NaN,Chuay Jai,0,8
5,Survivor: The Amazon,6,Jenna Morasca,96,Jenna,21,Bridgeville,Pennsylvania,ISTP,15,39,16,Sole Survivor,NaN,Jaburu,Jaburu,NaN,Jacaré,3,7
6,Survivor: Pearl Islands,7,Sandra Diaz-Twine,112,Sandra,29,Fort Lewis,Washington,ESTP,15,39,18,Sole Survivor,NaN,Drake,Drake,NaN,Balboa,0,3
7,Survivor: All-Stars,8,Amber Brkich,27,Amber,25,Beaver,Pennsylvania,ISFP,17,39,18,Sole Survivor,NaN,Chapera,Chapera,Chapera,Chaboga Mogo,6,6
8,Survivor: Vanuatu,9,Chris Daugherty,130,Chris,33,South Vienna,Ohio,ENTP,15,39,18,Sole Survivor,NaN,Lopevi,Lopevi,NaN,Alinta,3,6
9,Survivor: Palau,10,Tom Westman,150,Tom,40,Sayville,New York,ESTJ,15,39,20,Sole Survivor,NaN,Koror,NaN,NaN,Koror,0,12


**Q5:** Have any contestants won more than one time?  If so, select their records from the `sole_survivor` DataFrame, sorting the rows by season.  Save this as `Q5`.  If no contestant has won twice, save `Q5` as the string `None`.

In [8]:
#sole_survivor['full name'].value_counts()
multiWin = sole_survivor[(sole_survivor['full_name'] == 'Sandra Diaz-Twine') | (sole_survivor['full_name'] == 'Tony Vlachos')]
Q5 = multiWin.sort_values(by = 'season')
Q5

,season_name,season,full_name,castaway_id,castaway,age,city,state,personality_type,episode,day,order,result,jury_status,original_tribe,swapped_tribe,swapped_tribe_2,merged_tribe,total_votes_received,immunity_idols_won
6,Survivor: Pearl Islands,7,Sandra Diaz-Twine,112,Sandra,29,Fort Lewis,Washington,ESTP,15,39,18,Sole Survivor,NaN,Drake,Drake,NaN,Balboa,0,3
19,Survivor: Heroes vs. Villains,20,Sandra Diaz-Twine,112,Sandra,35,Fayetteville,North Carolina,ESTP,15,39,20,Sole Survivor,NaN,Villains,NaN,NaN,Yin Yang,3,4
27,Survivor: Cagayan,28,Tony Vlachos,424,Tony,39,Jersey City,New Jersey,ESTP,14,39,18,Sole Survivor,NaN,Aparri,Solana,NaN,Solarrion,5,6
39,Survivor: Winners at War,40,Tony Vlachos,424,Tony,45,Allendale,New Jersey,ESTP,15,39,22,Sole Survivor,NaN,Dakal,Dakal,NaN,Koru,0,9


**Q6:** What is the average age of contestants when they appeared on the show?  Save this as `Q6`.  Round to nearest integer.

In [9]:
Q6 = round(castaways['age'].mean(), 0).astype(int)
Q6

33

**Q7:** Who played the most total number of days of Survivor? If a contestant appeared on more than one season, you would add their total days for each season together. Save the top five contestants in terms of total days played as a DataFrame and call it `Q7`, sorted in descending order by total days played.  

The following columns should be included: `castaway_id`, `full_name`, and `total_days_played` where `total_days_played` is the sum of all days a contestant played. The index should go from 0 to n-1.

Note:  Be careful because on some seasons, the contestant was allowed to come back into the game after being voted off.  Take a look at [Season 23's contestant Oscar Lusth](https://en.wikipedia.org/wiki/Ozzy_Lusth#South_Pacific) in the `castaways` DataFrame as an example.  He was voted out 7th and then returned to the game.  He was then voted out 9th and returned to the game a second time.  He was then voted out 17th the final time.  Be aware of this in your calculations and make sure you are counting the days according to the last time they were voted off or won. 

In [48]:
dropReturn = castaways.drop_duplicates(['season', 'castaway_id'])
sumDays = dropReturn.groupby(['full_name', 'castaway_id'])['day'].sum().reset_index().sort_values(by = 'day',
                                                                                                  ascending = False)
sumDays = sumDays.rename(columns = {'day': 'total_days_played'})
sumDays = sumDays.reset_index(drop = True)
Q7 = sumDays[['castaway_id', 'full_name', 'total_days_played']].nlargest(5, 'total_days_played')
Q7

,castaway_id,full_name,total_days_played
0,55,Rob Mariano,131
1,197,Parvati Shallow,130
2,201,Oscar Lusth,128
3,179,Cirie Fields,121
4,112,Sandra Diaz-Twine,110


**Q8A & Q8B**: Using the `castaway_details` data, what is the percentage of total extroverts and introverts that have played the game (count players only once even if they have played in more than one season).  Do not count contestants without a personality type listed in your calculations.  Save these percentages as `Q8A` and `Q8B` respectively.  Note: Round all percentages to two decimal points and write as a float (example: 55.57).  

For more information on personality types check this [Wikipedia article](https://en.wikipedia.org/wiki/Myers%E2%80%93Briggs_Type_Indicator).

In [24]:
q8 = castaway_details.dropna(subset = ['personality_type'])
Q8A = round((((q8[q8['personality_type'].str.startswith("E")].shape[0]) / (q8.shape[0])) * 100), 2)
Q8B = round((((q8[q8['personality_type'].str.startswith("I")].shape[0]) / (q8.shape[0])) * 100), 2)
print("Extrovert %: ", Q8A)
print("Introvert %: ", Q8B)

Extrovert %:  53.63
Introvert %:  46.37


**Q9A & Q9B**: Now that we know the percentages of total players that are extroverted and introverted, let's see if that made a difference in terms of who actually won their season.

What is the percentage of total extroverts and introverts that have won the game (count players only once even if they have won more than one season)?  Save these percentages as `Q9A` and `Q9B` respectively.  Note: Round all percentages to two decimal points and write as a float (example: 55.57).

In [25]:
q9 = sole_survivor.drop_duplicates(subset = ['full_name'])
Q9A = round((((q9[q9['personality_type'].str.startswith("E")].shape[0]) / (q9.shape[0])) * 100), 2)
Q9B = round((((q9[q9['personality_type'].str.startswith("I")].shape[0]) / (q9.shape[0])) * 100), 2)
print("Extrovert %: ", Q9A)
print("Introvert %: ", Q9B)

Extrovert %:  61.54
Introvert %:  38.46


**Q10:** Which contestants have never received a tribal council vote (i.e. a vote to be voted out of the game as shown in the `vote_id` column in the `vote_history` DataFrame)? Note that there are various reasons for a contestant to not receive a tribal vote: they quit, made it to the end, medical emergency, etc.  Select their rows from the `castaway_details` DataFrame and save this as `Q10` in ascending order by `castaway_id`.  This should return a DataFrame and the index and missing values should be left as is.

In [13]:
x = pd.merge(castaway_details, castaways, left_on = 'castaway_id', right_on = 'castaway_id')

x = x.groupby(['full_name_x', 'castaway_id'])['total_votes_received'].sum().reset_index().sort_values(
    by = 'total_votes_received')
x
q10 = x[x['total_votes_received'] == 0].sort_values(by = 'castaway_id')
q10 = q10.rename(columns = {'full_name_x': 'full_name'})
q10 = q10[['castaway_id', 'full_name', 'total_votes_received']]

Q10 = castaway_details[(castaway_details['castaway_id'] == 22) | (castaway_details['castaway_id'] == 61) |
                      (castaway_details['castaway_id'] == 80) | (castaway_details['castaway_id'] == 131) |
                      (castaway_details['castaway_id'] == 132) | (castaway_details['castaway_id'] == 206) |
                      (castaway_details['castaway_id'] == 242) | (castaway_details['castaway_id'] == 283) |
                      (castaway_details['castaway_id'] == 314) | (castaway_details['castaway_id'] == 354) |
                      (castaway_details['castaway_id'] == 375) | (castaway_details['castaway_id'] == 385) |
                      (castaway_details['castaway_id'] == 412) | (castaway_details['castaway_id'] == 468) |
                      (castaway_details['castaway_id'] == 468) | (castaway_details['castaway_id'] == 537) |
                      (castaway_details['castaway_id'] == 540) | (castaway_details['castaway_id'] == 553) |
                      (castaway_details['castaway_id'] == 570)]
Q10.head()

,castaway_id,full_name,short_name,date_of_birth,date_of_death,gender,race,ethnicity,occupation,personality_type
21,22,Michael Skupin,Michael,1962-01-29,NaT,Male,NaN,NaN,Software Publisher;Part-Time Professional Speaker,ESFJ
60,61,Paschal English,Paschal,1945-03-05,NaT,Male,NaN,NaN,Judge,ISFJ
79,80,Brian Heidik,Brian,1968-03-09,NaT,Male,NaN,NaN,Used Car Salesman,ISTP
130,131,Jonathan Libby,Jonathan,1981-09-05,NaT,Male,NaN,NaN,Sales & Marketing Associate,ISTP
131,132,Wanda Shirk,Wanda,1949-08-24,NaT,Female,NaN,NaN,English Teacher,ENFP


**Q11:** What contestant has won the most number of challenges?  Select their row from the `castaway_details` DataFrame and save this as `Q11`.  This should return a DataFrame and the index and missing values should be left as is.

In [14]:
z = pd.merge(castaway_details, challenge_results, left_on = 'castaway_id', right_on = 'winner_id')
z = z.groupby(['full_name', 'castaway_id'])['winner_id'].count().reset_index().sort_values(by = 'winner_id',ascending = False)
z = z.nlargest(1, 'winner_id')
Q11 = castaway_details[castaway_details['castaway_id'] == 201]
Q11

,castaway_id,full_name,short_name,date_of_birth,date_of_death,gender,race,ethnicity,occupation,personality_type
200,201,Oscar Lusth,Ozzy,1981-08-23,NaT,Male,Mexican American,Hispanic or Latino,Waiter;Photographer,ISFP


**Q12:** Let's see how many winners ended up getting unanimous jury votes to win the game.  Create a Dataframe that shows the survivors that got unanimous jury votes with these columns in the final output: `season`, `season_name`, `winner_id`, `full_name`.  The DataFrame should be sorted by season and the index should go from 0 to n-1.  Save this as `Q12`. 

In [15]:
unamWinner = season_summary[(season_summary['final_vote'] == '9-0-0') | (season_summary['final_vote'] == '7-0') |
                           (season_summary['final_vote'] == '8-0-0') | (season_summary['final_vote'] == '10-0-0')]
Q12 = unamWinner[['season', 'season_name', 'winner_id', 'full_name']].reset_index(drop = True).sort_values(by = 'season')
Q12

,season,season_name,winner_id,full_name
0,14,Survivor: Fiji,221,Earl Cole
1,18,Survivor: Tocantins,281,James Thomas Jr.
2,26,Survivor: Caramoan,348,John Cochran
3,31,Survivor: Cambodia,433,Jeremy Collins
4,33,Survivor: Millennials vs. Gen X,498,Adam Klein


**Q13:** For the `castaway_details` DataFrame, there is a `full_name` column and a `short_name` column.  It would be helpful for future analysis to have the contestants first and last name split into separate columns.  First copy the `castaway_details` DataFrame to a new DataFrame called `Q13` so that we do not change the original DataFrame.  

Create two new columns and add the contestant's first name to a new column called `first_name` and their last name to a new column called `last_name`.  Add these columns to the `Q13` DataFrame.  Put the `first_name` and `last_name` columns between the `full_name` and `short_name` columns.

Note:  Be careful as some players have last names with multiple spaces.  For example, `Lex van den Berghe`.  You should code `Lex` as his first name and `van den Berghe` as his last name.

In [16]:
Q13 = castaway_details.copy()
Q13[['first_name', 'last_name']] = Q13['full_name'].str.split(n = 1, expand = True)
Q13 = Q13[['castaway_id', 'full_name', 'first_name', 'last_name', 'short_name', 'date_of_birth', 'date_of_death', 'gender',
       'race', 'ethnicity', 'occupation', 'personality_type']]
Q13.head()

,castaway_id,full_name,first_name,last_name,short_name,date_of_birth,date_of_death,gender,race,ethnicity,occupation,personality_type
0,1,Sonja Christopher,Sonja,Christopher,Sonja,1937-01-28,NaT,Female,NaN,NaN,Musician,ENFP
1,2,B.B. Andersen,B.B.,Andersen,B.B.,1936-01-18,2013-10-29,Male,NaN,NaN,Real Estate Developer,ESTJ
2,3,Stacey Stillman,Stacey,Stillman,Stacey,1972-08-11,NaT,Female,NaN,NaN,Attorney,ENTJ
3,4,Ramona Gray,Ramona,Gray,Ramona,1971-01-20,NaT,Female,Black,NaN,Biochemist/Chemist,ISTJ
4,5,Dirk Been,Dirk,Been,Dirk,1976-06-15,NaT,Male,NaN,NaN,Dairy Farmer,ISFP


**Q14:** Let's say that we wanted to predict a contestants personality type based on the information in the data files.  Your task is to create a DataFrame that lists the `castaway_id`, `full_name` and `personality_type` for each castaway contestant.  However, since most machine learning algorithms use numeric data, you want to change the personality types to the following numbers:
- ISTJ - 1
- ISTP - 2
- ISFJ - 3
- ISFP - 4
- INFJ - 5
- INFP - 6
- INTJ - 7
- INTP - 8
- ESTP - 9
- ESTJ - 10
- ESFP - 11
- ESFJ - 12
- ENFP - 13
- ENFJ - 14
- ENTP - 15
- ENTJ - 16
- Missing values - 17

Save this new DataFrame as `Q14` and sort based on `castaway_id` in ascending order.

In [37]:
Q14 = castaway_details.copy()
Q14 = Q14.replace({'ISTJ': 1, 'ISTP': 2, 'ISFJ': 3, 'ISFP': 4, 'INFJ': 5, 'INFP': 6, 'INTJ': 7, 'INTP': 8, 'ESTP': 9,
                   'ESTJ': 10, 'ESFP': 11, 'ESFJ': 12, 'ENFP': 13, 'ENFJ': 14, 'ENTP': 15, 'ENTJ': 16, np.nan: 17})
Q14['personality_type'] = Q14['personality_type'].astype(int)
Q14 = Q14[['castaway_id', 'full_name', 'personality_type']].sort_values(by = 'castaway_id')
Q14.head()

,castaway_id,full_name,personality_type
0,1,Sonja Christopher,13
1,2,B.B. Andersen,10
2,3,Stacey Stillman,16
3,4,Ramona Gray,1
4,5,Dirk Been,4


**Q15:** After thinking about this some more, you realize that you don't want to code the personality traits as you did in problem 14 since the data is not ordinal.  Some machine learning algorithms will assume that numbers close to each other are more alike than those that are away from each other and that is not the case with these personality types.

Let's create a new DataFrame called `Q15` that creates dummy columns (using `get_dummies`) for the original personality type column.  Add a prefix called "type" and drop the first column to help prevent multicollinearity.  The columns should be `castaway_id`, `full_name` followed by the various dummy columns for the personality types.  Don't worry about any missing values in this step.

Remember: Don't change any of the original DataFrames or CodeGrade will not work correctly for this assignment.  Make sure you use `copy()` if needed.

In [36]:
Q15 = castaway_details.copy()
Q15 = pd.get_dummies(Q15, columns = ['personality_type'], prefix = ['type'], drop_first = True)
Q15 = Q15.iloc[:, [0, 1, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]]
Q15.head()

,castaway_id,full_name,type_ENFP,type_ENTJ,type_ENTP,type_ESFJ,type_ESFP,type_ESTJ,type_ESTP,type_INFJ,type_INFP,type_INTJ,type_INTP,type_ISFJ,type_ISFP,type_ISTJ,type_ISTP
0,1,Sonja Christopher,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,B.B. Andersen,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,3,Stacey Stillman,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,Ramona Gray,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,5,Dirk Been,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


**Q16:** After running your data above through your machine learning model, you determine that a better prediction might come from breaking the personality type into its four parts (one part for each character in the type).  Your task is now to create a DataFrame called `Q16` that splits the personality type into the various parts and creates a new column for each part (these columns should be called `interaction` that will represent the first letter in the personality type, `information` for the second letter, `decision` for the third, and `organization` for the fourth).

Again, since most machine learning algorithms work with numeric data, perform the following on the four new columns:
- `interaction` --> code all `I`'s as `0` and `E`'s as `1`
- `information` --> code all `S`'s as `0` and `N`'s as `1`
- `decision` --> code all `T`'s as `0` and `F`'s as `1`
- `organization` --> code as `J`'s with `0` and `P`'s as `1`
- Any missing values should be coded with a `2`
- Double check that all of the above columns are an integer type.  Some students have a problem with CodeGrade because one of their columns ends up being a string instead of an int.

For example, if a contestant's personality type was `ENTJ`, your columns for that row would be:
- `1` for `interaction` because of the `E`
- `1` for `information` because of the `N`
- `0` for `decision` because of the `T` 
- `0` for `organization` because of the `J`

The new DataFrame should be sorted in `castaway_id` order and have the following columns in this order: `castaway_id`, `full_name`, `personality_type`, `interaction`, `information`, `decision`, `organization`.

Remember: Don't change any of the original DataFrames or CodeGrade will not work correctly for this assignment.  Make sure you use `copy()` if needed.

In [31]:
Q16 = castaway_details.copy()
Q16['interaction'] = Q16['personality_type'].str[0].replace({'E': 1, 'I': 0, np.nan: 2}).astype(int)
Q16['information'] = Q16['personality_type'].str[1].replace({'N': 1, 'S': 0, np.nan: 2}).astype(int)
Q16['decision'] = Q16['personality_type'].str[2].replace({'F': 1, 'T': 0, np.nan: 2}).astype(int)
Q16['organization'] = Q16['personality_type'].str[3].replace({'P': 1, 'J': 0, np.nan: 2}).astype(int)
Q16 = Q16[['castaway_id', 'full_name', 'personality_type', 'interaction', 'information',
           'decision', 'organization']].sort_values(by = 'castaway_id')
Q16.head()

,castaway_id,full_name,personality_type,interaction,information,decision,organization
0,1,Sonja Christopher,ENFP,1,1,1,1
1,2,B.B. Andersen,ESTJ,1,0,0,0
2,3,Stacey Stillman,ENTJ,1,1,0,0
3,4,Ramona Gray,ISTJ,0,0,0,0
4,5,Dirk Been,ISFP,0,0,1,1


**Q17:** Using data from `castaways`, create a DataFrame called `Q17` that bins the contestant ages (their age when they were on the season, not their current age) into the following age categories:
- 18-24
- 25-34
- 35-44
- 45-54
- 55-64
- 65+

The final DataFrame should have the following columns in this order: `season`, `castaway_id`, `full_name`, `age`, and `age_category`.  The DataFrame should be sorted by age and then castaway_id.  The index should be 0 through n-1.  You should have the same amount of rows as in the `castaways` DataFrame.

Remember: Don't change any of the original DataFrames or CodeGrade will not work correctly for this assignment.  Make sure you use `copy()` if needed.

In [20]:
Q17 = castaways.copy()
Q17['age_category'] = Q17['age']
Q17.loc[:, 'age_category'] = pd.cut(x = Q17['age_category'], bins = [Q17['age'].min() - 1, 24, 34, 44, 54, 64, Q17['age'].max()],
                                   labels = ['18-24', '25-34', '35-44', '45-54', '55-64', '65+'])
Q17 = Q17[['season', 'castaway_id', 'full_name', 'age', 'age_category']].sort_values(by = ['age', 'castaway_id'])
Q17 = Q17.reset_index(drop = True)
Q17.head()

,season,castaway_id,full_name,age,age_category
0,33,491,Will Wahl,18,18-24
1,36,528,Michael Yerger,18,18-24
2,18,270,Spencer Duhm,19,18-24
3,22,336,Natalie Tenerelli,19,18-24
4,23,350,Brandon Hantz,19,18-24


**Q18:** Based on the age categories you created above, what are the normalized percentages for the various age categories using `value_counts()`.  Sort the value counts by index.  Save this as `Q18`.

In [21]:
Q18 = Q17['age_category'].value_counts(normalize = True).sort_index()
Q18

18-24    0.190289
25-34    0.437008
35-44    0.211286
45-54    0.124672
55-64    0.031496
65+      0.005249
Name: age_category, dtype: float64

**Q19:** Which contestant(s) played a perfect game?  A perfect game is considered when the contestant:
- didn't receive any tribal council votes all season (this is different than Q10 since some players played multiple times.  They got voted out in one season so they would not show in Q10 but they came back for another season and didn't receive any tribal council votes)
- won the game
- got unanimous jury votes (see question 12)

Save this DataFrame as `Q19` with the following columns: `season_name`, `season`, `castaway_id`, `full_name`, `tribal_council_votes`, `jury_votes`.  The DataFrame should be sorted by season and the index should be 0 to n-1.

In [26]:
castMerge = pd.merge(castaway_details, castaways, left_on = 'castaway_id', right_on = 'castaway_id')
castMerge = castMerge.groupby(['season', 'full_name_x', 'castaway_id'])['total_votes_received'].sum().reset_index()
castMerge = castMerge.rename(columns = {'full_name_x': 'full_name'})
castMerge = castMerge[castMerge['total_votes_received'] == 0]
unamJurVote = season_summary[(season_summary['final_vote'] == '9-0-0') | (season_summary['final_vote'] == '7-0') |
                           (season_summary['final_vote'] == '8-0-0') | (season_summary['final_vote'] == '10-0-0')]
Q19 = pd.merge(castMerge, unamJurVote, left_on = 'castaway_id', right_on = 'winner_id')
Q19 = Q19.rename(columns = {'season_x': 'season', 'full_name_x': 'full_name', 'total_votes_received': 'tribal_council_votes',
                           'final_vote': 'jury_votes'})
Q19['jury_votes'] = Q19['jury_votes'].str[0]
Q19 = Q19[['season_name', 'season', 'castaway_id', 'full_name', 'tribal_council_votes', 'jury_votes']]
Q19

,season_name,season,castaway_id,full_name,tribal_council_votes,jury_votes
0,Survivor: Tocantins,18,281,James Thomas Jr.,0,7
1,Survivor: Caramoan,26,348,John Cochran,0,8
